<a href="https://colab.research.google.com/github/mthbrito/Formata-dados-Hidroweb/blob/main/formata_dados_pluviometricos_do_hidroweb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Obs.: Esse código formata apenas os dados de CHUVA do Hidroweb**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import os
local = '/content/drive/MyDrive/Dados Pluviométricos/Postos CSV'
postos = os.listdir(local)
print(postos)

['838002_Chuvas.csv', '737023_Chuvas.csv', '839019_Chuvas.csv', '840015_Chuvas.csv', '838031_Chuvas.csv', '838021_Chuvas.csv', '839009_Chuvas.csv', '737025_Chuvas.csv', '737026_Chuvas.csv', '737027_Chuvas.csv', '737029_Chuvas.csv', '737030_Chuvas.csv', '737031_Chuvas.csv', '737032_Chuvas.csv', '737033_Chuvas.csv', '737034_Chuvas.csv', '737036_Chuvas.csv', '737037_Chuvas.csv', '738027_Chuvas.csv', '738028_Chuvas.csv', '738029_Chuvas.csv', '738030_Chuvas.csv', '738035_Chuvas.csv', '738036_Chuvas.csv', '739021_Chuvas.csv', '739022_Chuvas.csv', '739023_Chuvas.csv', '739025_Chuvas.csv', '739027_Chuvas.csv', '739028_Chuvas.csv', '739029_Chuvas.csv', '739030_Chuvas.csv', '739031_Chuvas.csv', '739032_Chuvas.csv', '739034_Chuvas.csv', '740014_Chuvas.csv', '740015_Chuvas.csv', '740016_Chuvas.csv', '740017_Chuvas.csv', '740018_Chuvas.csv', '740019_Chuvas.csv', '740020_Chuvas.csv', '740021_Chuvas.csv', '740023_Chuvas.csv', '740024_Chuvas.csv', '740025_Chuvas.csv', '740029_Chuvas.csv', '837013_Chuv

In [ ]:
postos_pluviometricos = [local+'/'+i for i in postos]
print(postos_pluviometricos)

['/content/drive/MyDrive/Dados Pluviométricos/Postos CSV/838002_Chuvas.csv', '/content/drive/MyDrive/Dados Pluviométricos/Postos CSV/737023_Chuvas.csv', '/content/drive/MyDrive/Dados Pluviométricos/Postos CSV/839019_Chuvas.csv', '/content/drive/MyDrive/Dados Pluviométricos/Postos CSV/840015_Chuvas.csv', '/content/drive/MyDrive/Dados Pluviométricos/Postos CSV/838031_Chuvas.csv', '/content/drive/MyDrive/Dados Pluviométricos/Postos CSV/838021_Chuvas.csv', '/content/drive/MyDrive/Dados Pluviométricos/Postos CSV/839009_Chuvas.csv', '/content/drive/MyDrive/Dados Pluviométricos/Postos CSV/737025_Chuvas.csv', '/content/drive/MyDrive/Dados Pluviométricos/Postos CSV/737026_Chuvas.csv', '/content/drive/MyDrive/Dados Pluviométricos/Postos CSV/737027_Chuvas.csv', '/content/drive/MyDrive/Dados Pluviométricos/Postos CSV/737029_Chuvas.csv', '/content/drive/MyDrive/Dados Pluviométricos/Postos CSV/737030_Chuvas.csv', '/content/drive/MyDrive/Dados Pluviométricos/Postos CSV/737031_Chuvas.csv'

In [ ]:
def le_dados(dado):
    """"Lê os arquivos csv que estão com problemas na formatação"""
    with open(dado, 'r', encoding='latin1') as arquivo:
      conteudo = arquivo.read()
    return conteudo

def separa_dados(inicio, dado):
    """Separa o conteúdo desejado a partir do início da palavra desejada"""
    conteudo = []
    for i in dado[dado.find(inicio):].split(sep='\n'):
      conteudo.append(i.split(sep=';'))
    return conteudo

def zipa_dados(dado):
  """Zipa todas as listas aninhadas"""
  from itertools import zip_longest
  return list(zip_longest(*dado))

def dados2df(dado):
  """Converte os dados para dicionário e em seguida para dataframe do Pandas"""
  dados = {i[0]:i[1:] for i in dado}
  df = pd.DataFrame(dados)
  return df

def seleciona_colunas(dado):
  """Seleciona as colunas de interesse do dataframe"""
  from itertools import chain
  colunas = list(chain(['EstacaoCodigo','Data'],['Chuva'+str(i).rjust(2,'0') for i in range(1,32)]))
  return dado[colunas]

def converte_dados(dado):
  """Converte as strings presentes no dataframe em floats e datas"""
  from datetime import date
  dado = dado.applymap(lambda x: x.replace(',', '.').replace('"','') if x != None else '')
  dado.iloc[:,2:] = dado.iloc[:,2:].applymap(lambda x:float(x) if x != '' else None)
  return dado

def organiza_dados(dado):
  """Organiza os dados gerados, concatenando-os e formatando-os"""
  from functools import reduce
  dfs = []
  for index, row in dado.iterrows():
    if row[1] != None:
      df = pd.DataFrame({'data':[f'{i}/{row[1][3:]}' for i in range(1,32)],'mm':list(row[2:])})
    dfs.append(df)
  df_concat = reduce(lambda left, right: pd.concat([left,right]),dfs)

  from datetime import date
  datas = []
  for index, row in df_concat.iterrows():
    data = row[0].split(sep='/')
    if len(data) == 3:
      try:
        datas.append(date(int(data[2]),int(data[1]),int(data[0])))
      except:
        datas.append(None)
    else:
      datas.append(None)
  df_concat['data'] = datas

  df_sorted = df_concat.sort_values(by='data').reset_index()
  df = df_sorted[df_sorted['data'].notnull()]
  df = df[['data','mm']]
  return df

def formata_dados(dado):
  """Realiza a formatacão dos dados de chuva do Hidroweb, retornando um dataframe organizado"""
  resultado = organiza_dados(converte_dados(seleciona_colunas(dados2df(zipa_dados(separa_dados('EstacaoCodigo',le_dados(dado)))))))
  return resultado

In [ ]:
resultado = formata_dados(postos_pluviometricos[100])
resultado.head(5)

,data,mm
0,1962-09-01,0.0
1,1962-09-02,0.0
2,1962-09-03,0.0
3,1962-09-04,0.0
4,1962-09-05,0.0


In [ ]:
%%time

import re
from tqdm import tqdm
nomes_arquivos = [re.findall(r'\d+(?=_)', i)[0] for i in postos if i.endswith('_Chuvas.csv')]

for i in tqdm(range(len(postos))):
    try:
        # Tentar formatar os dados do posto pluviométrico
        resultado = formata_dados(postos_pluviometricos[i])
        out = '/content/drive/MyDrive/Dados Pluviométricos/Postos CSV formatados/'+nomes_arquivos[i]+'.csv'
        if os.path.exists(out):
          print(f'Arquivo{nomes_arquivos[i]} já existe!')
        else:
          resultado.to_csv(out)
          print(f'Arquivo {nomes_arquivos[i]}.csv foi salvo.')
    except Exception as e:
        # Se ocorrer um erro, imprimir uma mensagem de erro
        print(f'Erro no arquivo {i}: {e}')
        if os.path.exists(out):
          print(f'Arquivo {nomes_arquivos[i]} já existe!')

  1%|          | 1/117 [00:05<10:17,  5.32s/it]

Arquivo838002 já existe!


  2%|▏         | 2/117 [00:14<14:02,  7.32s/it]

Arquivo737023 já existe!


  3%|▎         | 3/117 [00:17<10:38,  5.60s/it]

...

Arquivo840003 já existe!


 99%|█████████▉| 116/117 [03:13<00:02,  2.27s/it]

Arquivo840010 já existe!


100%|██████████| 117/117 [03:14<00:00,  1.66s/it]

Arquivo839021 já existe!
CPU times: user 3min 6s, sys: 1.31 s, total: 3min 8s
Wall time: 3min 14s
